In [10]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
from typing import Dict

import gurobipy as gp
from gurobipy import GRB
from itertools import product

In [11]:
filepath = '../data/US_latest_data_20220428.parquet' 
return_filepath = '../data/Exp_Return_Mom.parquet'
df = pd.read_parquet(filepath, engine = 'pyarrow')
expected_return = pd.read_parquet(return_filepath, engine = 'pyarrow')

In [12]:
def clean_stock_data(df: pd.DataFrame) -> pd.DataFrame:
    # select required months
    df = df[["DateYYYYMM","ws_id","Price","Div","mom","mktcap","median_volume_usd"]]
    df = df.rename(columns={"DateYYYYMM": "date", "ws_id":"asset", "Price":"price", "Div":"div",
           "mom":"factor"})
    
    # Correct datetime 
    df['date'] = pd.to_datetime(df['date'], format = "%Y%m")
    
    # Change units for dollar valued data
    df['mktcap'] = df['mktcap'] *1000000
    df['median_volume_usd'] = df['median_volume_usd'] *1000000
    
    # replace missing values
    df[['price', 'div']] = df[['price', 'div']].fillna(0)
    
    return df

def clean_return_data(df: pd.DataFrame) -> pd.DataFrame:
    df = df.rename(columns={"Date": "date", "Asset":"asset", "Exp_Return":"exp_return"})
    df['date'] = pd.to_datetime(df['date'], format = "%Y%m")
    
    return df

In [13]:
df = clean_stock_data(df)
expected_return = clean_return_data(expected_return)

In [14]:
# testing 
df_time = df[df.date == df.date.min() + pd.DateOffset(months=1)]
df_by_mktcap = df_time.sort_values(by = 'mktcap', ascending=False).reset_index(drop=True)[:1000]
df_by_factor = df_by_mktcap.sort_values(by = 'factor', ascending=False).reset_index(drop=True)[:250] 

In [15]:
def filter_eligible_investments(df: pd.DataFrame, date: pd.Timestamp, ratio: float = 0.25, size: int = 1000) -> pd.DataFrame:
    """
    Return the dataframe containing the top ratio% assets based on the factor of the top 1000 assets  of top asset based on market cap.
    Remove assets that no longer exist 1 period ahead
    """
    factor_num = int(ratio*size)
    # the above is for target, but for actual we allow for the assets we already own
    # for actual, if an asset is not in the top 25% and you dont own it you cannot buy it. 
    
    df_current = df[df['date'] == date]
    df_next = df[df['date'] == date + pd.DateOffset(months=1)]
    
    asset_diff = set(df_current['asset']) - set(df_next['asset'])
    df_current = df_current[~df_current['asset'].isin(asset_diff)]
    
    df_by_mktcap = df_current.sort_values(by = 'mktcap', ascending=False).reset_index(drop=True)[:size]
    df_by_factor = df_by_mktcap.sort_values(by = 'factor', ascending=False).reset_index(drop=True)[:factor_num] 
     
    return df_by_factor

def update_assets(df: pd.DataFrame, valuation: float) -> Dict:
    """
    Return the dictionary of asset_names as the key and price/quantity tuple as the value. The proportion how much to purchase is determined by the  
    """
    
    df['ratio'] = df['mktcap']/df['mktcap'].sum()
    df['quantity'] = np.floor(df['ratio']*valuation/df['price'])
    assets = dict(zip(df['asset'], zip(df['quantity'], df['price'])))
    return assets
    

In [16]:
df1 = df[df.date == pd.to_datetime("197201", format = "%Y%m")]
df2 = df[df.date == pd.to_datetime("197202", format = "%Y%m")]

In [33]:
target_portfolio = {
    pd.to_datetime("197112", format = "%Y%m"): {
        'valuation': 10e8, # CASH+SHARES*PRICE + DIV
        'cash': 10e8,
        'div_paid': 0, #keep for debugging
        # 'index': 1, #Will be added back in if we consider cash injections
        'asset': {}, # {asset: (0,1) for asset in df.asset.unique()},
        # 'trade_cost': 0,
        # 'turnover': 0,
        # 'cash_flow': 0
    },
}


for date in pd.to_datetime(df['date'].sort_values().unique()):
    # filter the dataframe to the current date 
    df_current = df[df['date'] == date]
    
    # initialise current date key in portfolio
    target_portfolio[date] = {}
    
    # update dividents paid
    prev_assets = target_portfolio[date-pd.DateOffset(months=1)]['asset'] 
    
    div_paid = 0
    asset_value = 0
    for asset_name, quantity_price in prev_assets.items():
        if asset_name not in df_current.asset.to_list():
            asset_value += quantity_price[0]*quantity_price[1]
        else:
            asset_value += float(quantity_price[0] * df_current.loc[df_current['asset'] == asset_name, 'price'])
            div_paid += float(quantity_price[0]*df_current.loc[df_current['asset'] == asset_name, 'div'])

    
    target_portfolio[date]['div_paid'] = div_paid
    target_portfolio[date]['valuation'] = asset_value + div_paid + target_portfolio[date - pd.DateOffset(months=1)]['cash'] 
    
    # print(f"{date}: {target_portfolio[date]['valuation']}")
    
    # subset to eligible assets names
    df_current = filter_eligible_investments(df, date)
    
    # update the assets to the preferred ones
    target_portfolio[date]['asset'] = update_assets(df_current, target_portfolio[date]['valuation'])
    
    # update cash amount
    target_portfolio[date]['cash'] = target_portfolio[date]['valuation'] - sum([quantity_price[0]*quantity_price[1] for asset_name, quantity_price in target_portfolio[date]['asset'].items()])

    

In [32]:
test = pd.DataFrame.from_dict(target_portfolio[pd.to_datetime("197202", format = "%Y%m")]['asset'], orient='index')

In [34]:
target_portfolio[pd.to_datetime("202012", format = "%Y%m")]#['valuation'] # valutation at dec 2020 should be 70037236032 a return of 14.28% calculate by (val_end/val_1)^(12/n)

{'div_paid': 223062798.8695383,
 'valuation': 707215281693.1654,
 'asset': {},
 'cash': 707215281693.1654}

## Optimisation 

In [55]:
def calculate_large_implicit_cost(monthly_portfolio: Dict, df_monthly: pd.DataFrame) -> Dict:
    """
    Calculate the implicit cost of transacting each asset. This formulation of the implicit cost follows the formula,
    6*valuation/market_cap
    This is the implicit cost used for larger transactions
    """
    
    valuation = monthly_portfolio['valuation']
    df_monthly.loc[:,'large_implicit_cost'] = 6*valuation/df_monthly['mktcap']
    
    implicit_cost = dict(zip(df_monthly['asset'], df_monthly['large_implicit_cost']))
    
    return implicit_cost


def calculate_small_implicit_cost(monthly_portfolio: Dict, df_monthly: pd.DataFrame) -> Dict:
    """
    Calculate the implicit cost of transacting each asset. This formulation of the implicit cost follows the formula,
    bid_ask_spread/valuation
    This is the implicit cost used for smaller transactions
    """
    
    df_monthly.loc[:,'small_implicit_cost'] = 0.0013 * np.sqrt(np.median(df_monthly['mktcap'])/df_monthly['mktcap'])
    
    implicit_cost = dict(zip(df_monthly['asset'], df_monthly['small_implicit_cost']))
    
    return implicit_cost    

def get_asset_set(df:pd.DataFrame, date:pd.Timestamp) -> list:
    """
    Get the set of assets available in the current month and in the next month
    """
    df_current = filter_eligible_investments(df, date)
    df_next = filter_eligible_investments(df, date + pd.DateOffset(months=1))
    
    asset_diff = set(df_next['asset']) - set(df_current['asset'])
    assets = df_current['asset'].to_list()
    
    for asset in asset_diff:
        assets.append(asset)
     
    return assets

#### Data:
* $A$: set of assets


* $m_j$ = expected return for asset $j\in A$ 
* $u_j$ = implicit cost lower bound for trading large proportions of asset $j\in A$   
* $v_j$ = implicit cost lower bound for trading small proportions of asset $j\in A$   
* $s$ = explicit cost lower bound for trading large proportions of asset $j\in A$   
* $t$ = explicit cost lower bound for trading small proportions of asset $j\in A$   
* $a_j$ = pre-trade proportion of the portfolio made up by asset $j\in A$
* $e$ = small value 
* $M$ = large value


#### Variables:
* $w_j$: weight of trade for stock $j\in A$
* $c_{Ej}$: explicit cost of trading asset $j\in A$ 
* $c_{Ij}$: implicit cost of trading asset $j\in A$ 
* $x_j$: binary variable, taking value $1$ if the weight of trade for stock $j\in A$ is $0$ and $1$ otherwise.

#### Model:

Max $z = \sum_{j\in A} m_j w_j - c_{Ej} - c_{Ij}$

subject to 
* $c_{Ej} \geq u_j w_j^2, \quad \forall j\in A$ 
* $c_{Ej} \geq v_j w_j, \quad \forall j\in A$ 
* $c_{Ej} \geq -v_j w_j, \quad \forall j\in A$ 
* $c_{Ij} \geq s w_j, \quad \forall j\in A$ 
* $c_{Ij} \geq -s w_j, \quad \forall j\in A$ 
* $c_{Ij} \geq t x_j, \quad \forall j\in A$ 
* $w_j \geq e - M(1-x_j), \quad \forall j \in A$
* $w_j \leq -e + M x_j, \quad \forall j \in A$
* $\sum_{j\in A} w_j = 0$
* $w_j \geq -a_j, \quad \forall j \in A$
* $w_j \in [-1,1], x_j \in \{0,1\}, c_{Ij}, c_{Ej} \geq 0, \quad \forall j \in A$
 

In [57]:
# Obtain data
date = pd.to_datetime("202008", format = "%Y%m")

df_current = df[df.date == date]
monthly_return = expected_return[expected_return['date'] == date + pd.DateOffset(months=1)]
## update asset_dict to have values of one step ahead
A = get_asset_set(df, date)
u = calculate_large_implicit_cost(target_portfolio[date], df_current)
v = calculate_small_implicit_cost(target_portfolio[date], df_current)
s = 0.0001
t = 5/target_portfolio[date]['valuation']
mu = dict(zip(monthly_return['asset'], monthly_return['exp_return']))
M = 10e8
e = 10e-8
a = {}

for asset_name in A:
    if asset_name in target_portfolio[date]['asset'].keys():
        a[asset_name] = target_portfolio[date]['asset'][asset_name][0] * target_portfolio[date]['asset'][asset_name][1]/target_portfolio[date]['valuation'] 
    else: 
        a[asset_name] = 0

# Initialise Model
m = gp.Model('Optimisation')

# Define variables
w = m.addVars(A, vtype = GRB.SEMICONT, lb=-1, ub=1, name='w')
x = m.addVars(A, vtype = GRB.BINARY, name ='x')
ce = m.addVars(A, vtype = GRB.CONTINUOUS, name='ce')
ci = m.addVars(A, vtype = GRB.CONTINUOUS, name='ci')

# Define objective
m.setObjective(gp.quicksum(mu[j]*w[j] - ce[j] - ci[j] for j in A if j in mu.keys()),GRB.MAXIMIZE) # bandaid fix MUST return to this

# Define constraints
m.addConstrs( ci[j] >= u[j]*w[j]**2 for j in A)
m.addConstrs( ci[j] >= v[j]*w[j] for j in A)
m.addConstrs( ci[j] >= -v[j]*w[j] for j in A)

m.addConstrs( ce[j] >= s*w[j] for j in A)
m.addConstrs( ce[j] >= -s*w[j] for j in A)
m.addConstrs( ce[j] >= t*x[j] for j in A)

m.addConstrs(w[j] >= e - M*(1-x[j]) for j in A)
m.addConstrs(w[j] <= -e + M*x[j] for j in A)

m.addConstr(gp.quicksum(w[j] for j in A) == 0)
m.addConstrs(w[j] >= -a[j] for j in A)

# Solve and display
m.optimize()

for v in m.getVars():
    if v.x > 1e-6 or v.x < -1e-6:
        print(v.varName, v.x)

print('Total value: ', m.objVal)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 2433 rows, 1216 columns and 4864 nonzeros
Model fingerprint: 0xf60408db
Model has 304 quadratic constraints
Variable types: 608 continuous, 304 integer (304 binary)
Semi-Variable types: 304 continuous, 0 integer
Coefficient statistics:
  Matrix range     [8e-12, 1e+09]
  QMatrix range    [2e+00, 7e+03]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [4e-03, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-07, 1e+09]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 1405 rows and 184 columns
Presolve time: 0.02s
Presolved: 1028 rows, 1032 columns, 2358 nonzeros
Presolved model has 304 quadratic constraint(s)
Variable types: 782 continuous, 250 integer (250 binary)

Root relaxation: objective 2.494112e-02, 56 iterations, 0.00 seconds (0.00 

/tmp/ipykernel_33623/1698262193.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_monthly.loc[:,'large_implicit_cost'] = 6*valuation/df_monthly['mktcap']
/tmp/ipykernel_33623/1698262193.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_monthly.loc[:,'small_implicit_cost'] = 0.0013 * np.sqrt(np.median(df_monthly['mktcap'])/df_monthly['mktcap'])


H    0     0                       0.0000484    0.01086      -     -    0s
H    0     0                       0.0000492    0.00803      -     -    0s
H    0     0                       0.0000497    0.00718      -     -    0s
H    0     0                       0.0000498    0.00718      -     -    0s
H    0     0                       0.0000504    0.00708      -     -    0s
H    0     0                       0.0000509    0.00705      -     -    0s
H    0     0                       0.0000514    0.00694      -     -    0s
H    0     0                       0.0000516    0.00679      -     -    0s
H    0     0                       0.0000516    0.00659      -     -    0s
H    0     0                       0.0000526    0.00646      -     -    0s
*    0     0               0       0.0004809    0.00048  0.00%     -    0s

Explored 1 nodes (2126 simplex iterations) in 0.63 seconds (0.22 work units)
Thread count was 12 (of 12 available processors)

Solution count 10: 0.000480915 5.25782e-05 5.16